In [27]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd

# Scrape news title and paragraph from NASA Mars News

In [3]:
# Set Executable Path & Initialize Chrome Browser
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

In [4]:
# Visit the NASA Mars News Site
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [5]:
# Parse Results HTML with BeautifulSoup
# Find Everything Inside:
#   <ul class="item_list">
#     <li class="slide">

html = browser.html
news_soup = bs(html, "html.parser")
slide_element = news_soup.select_one("ul.item_list li.slide")

In [6]:
slide_element.find("div", class_="content_title")

<div class="content_title"><a href="/news/8678/the-detective-aboard-nasas-perseverance-rover/" target="_self">The Detective Aboard NASA's Perseverance Rover</a></div>

In [7]:
# Scrape the Latest News Title
# Use Parent Element to Find First <a> Tag and Save it as news_title
news_title = slide_element.find("div", class_="content_title").get_text()
print(news_title)

The Detective Aboard NASA's Perseverance Rover


In [8]:
# Scrape the Latest Paragraph Text
news_paragraph = slide_element.find("div", class_="article_teaser_body").get_text()
print(news_paragraph)

An instrument called SHERLOC will, with the help of its partner WATSON, hunt for signs of ancient life by detecting organic molecules and minerals.


# Scrape Featured Image from JPL Mars Space Images

In [9]:
# Visit the NASA JPL (Jet Propulsion Laboratory) Site
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

# Visit the NASA JPL site
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

In [10]:
# Ask Splinter to Go to Site and Click Button with Class Name full_image
# <button class="full_image">Full Image</button>
full_image_button = browser.find_by_id("full_image")
full_image_button.click()

In [11]:
# Find "More Info" Button and Click It
browser.is_element_present_by_text("more info", wait_time=1)
more_info_element = browser.find_link_by_partial_text("more info")
more_info_element.click()

In [12]:
# Parse Results HTML with BeautifulSoup
html = browser.html
image_soup = bs(html, "html.parser")

In [13]:
img_url = image_soup.select_one("figure.lede a img").get("src")
img_url

'/spaceimages/images/largesize/PIA18826_hires.jpg'

In [14]:
# Use Base URL to Create Absolute URL
featured_image_url = f"https://www.jpl.nasa.gov{img_url}"
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18826_hires.jpg'

# Scrape Mars Weather 

In [42]:
# Visit the Mars Weather Twitter Account
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [43]:
# Parse Results HTML with BeautifulSoup
weather_html = browser.html
weather_soup = bs(weather_html, "html.parser")

In [44]:
# Find a Tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_soup.find("p", attrs= {"class": "tweet",
                                                    "data-name": "Mars Weather"
                                                        })
mars_weather_tweet

In [48]:
# Search Within Tweet for <p> Tag Containing Tweet Text
mars_weather = weather_soup.find("div", "css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0").get_text()
mars_weather

AttributeError: 'NoneType' object has no attribute 'get_text'

# Scrape Mars Facts

In [21]:
# Visit the Mars Facts Site Using Pandas to Read
mars_df = pd.read_html("https://space-facts.com/mars/")[0]
mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [22]:
mars_df.columns=["Description", "Value"]
mars_df.set_index("Description", inplace=True)
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [23]:
# Visit the USGS Astrogeology Science Center Site
executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

In [24]:
hemisphere_image_urls = []

# Get a list of all the Hemispheres
links = browser.find_by_css("a.product-item h3")
for item in range(len(links)):
    hemisphere = {}
    
    # Find element on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[item].click()
    
    # Find sample image anchor tag & extract <href>
    sample_element = browser.find_link_by_text("Sample").first
    hemisphere["img_url"] = sample_element["href"]
    
    # Get hemisphere title
    hemisphere["title"] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Navigate back
    browser.back()

In [25]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]